# Create query-opinion pairs of train & validation data for finetuning

1. I used the BERT tokenizer for ease of convenience (even though we will use numerous non-BERT models)
2. I set the max_tokens to 7800 to account for the tokenization differences between the different models (for 8192 context window)
3. I used GPT 4o to generate both relevant and irrelevant queries
4. Since most of the opinions are above the 8192 context window, the opinions are first chunks to max-tokens and then query pairs are generated for each chunk

# Import libraries

In [1]:
import pandas as pd

import finetune_utils as util

PATH = ""

# Load the data

In [2]:
cols = ["opinion_id", "opinion", "opinion_word_count"]

train = pd.read_csv(f"{PATH}outputs/4.finetune_train.csv")
train = train[cols]

val = pd.read_csv(f"{PATH}outputs/4.finetune_val.csv")
val = val[cols]

# Take a look at the word count distribution & split train/val opinions to chunks

In [3]:
train["opinion_word_count"].describe()

count      315.000000
mean      2288.469841
std       2606.012431
min         52.000000
25%        613.000000
50%       1533.000000
75%       3026.000000
max      21186.000000
Name: opinion_word_count, dtype: float64

In [4]:
val["opinion_word_count"].describe()

count      93.000000
mean     1426.150538
std      1474.447332
min        55.000000
25%       395.000000
50%       994.000000
75%      1863.000000
max      8475.000000
Name: opinion_word_count, dtype: float64

since we have long context records but the base models we use for finetuning have shorter context windows, to best utilize our train/val dataset, we will split the opinions to chunks based on token counts, generate queries for each chunk and use the chunked data for finetuning.

## Chunk the opinions

In [5]:
train = util.process_opinions(train)
train.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (719 > 512). Running this sequence through the model will result in indexing errors


,chunk_id,chunk_size,chunked_opinion,opinion_id,opinion
0,4200405_1,197,DISTRICT COURT OF APPEAL OF THE STATE OF FLORI...,4200405,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...
1,2790710_1,1067,Twersky v Incorporated Vil. of Great Neck ( 20...,2790710,Twersky v Incorporated Vil. of Great Neck ( 2...
2,1470544_1,2286,951 A. 2d 180 ( 2008 ) Philip S. HORNER v. GOV...,1470544,951 A.2d 180 (2008) Philip S. HORNER v. GOVERN...
3,1083484_1,7760,IN THE COURT OF CRIMINAL APPEALS OF TENNESSEE ...,1083484,IN THE COURT OF CRIMINAL APPEALS OF TENNESSEE...
4,1083484_2,4820,"207 ). As pointed out above, the applicable st...",1083484,IN THE COURT OF CRIMINAL APPEALS OF TENNESSEE...


In [6]:
len(train)

351

In [7]:
train["chunk_size"].describe()

count     351.000000
mean     3019.421652
std      2468.817057
min        68.000000
25%       936.000000
50%      2303.000000
75%      4599.500000
max      7800.000000
Name: chunk_size, dtype: float64

In [8]:
val = util.process_opinions(val)
val.head()

,chunk_id,chunk_size,chunked_opinion,opinion_id,opinion
0,6237262_1,674,Mr. Justice Mercur delivered the opinion of th...,6237262,Mr. Justice Mercur delivered the opinion of t...
1,1467030_1,2337,217 N. J. Super. 541 ( 1987 ) 526 A. 2d 290 AL...,1467030,217 N.J. Super. 541 (1987) 526 A.2d 290 ALAN C...
2,5239595_1,1603,"Howard, J. : By the ' will of Byron S. Briggs,...",5239595,"Howard, J.: By the'will of Byron S. Briggs, w..."
3,6198051_1,410,"Alfred M. Lama, J. In this case, the defendant...",6198051,"Alfred M. Lama, J. In this case, the defendan..."
4,3531217_1,3227,The purpose of this proceeding is twofold. Fir...,3531217,The purpose of this proceeding is twofold. Fir...


In [9]:
len(val)

95

In [10]:
val["chunk_size"].describe()

count      95.000000
mean     1809.242105
std      1663.952822
min        79.000000
25%       578.500000
50%      1405.000000
75%      2454.500000
max      7772.000000
Name: chunk_size, dtype: float64

# Make GPT calls on train data

In [11]:
%%time

train_pred = util.predict(train)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 3
INFO:root:Completed: 1
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 0
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 4
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 5
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 7
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 6
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 O

CPU times: user 16.6 s, sys: 912 ms, total: 17.5 s
Wall time: 1min 51s


In [12]:
train_queries = pd.DataFrame(train_pred)
train_queries.head()

,model,input_tokens,output_tokens,relevant,irrelevant,chunk_id
0,gpt-4o-2024-11-20,7965,41,What are the legal standards for withdrawing a...,What are the tax implications of starting a sm...,1083484_1
1,gpt-4o-2024-11-20,1211,43,What legal principles determine a property own...,What are the tax implications of selling a pro...,2790710_1
2,gpt-4o-2024-11-20,2492,48,What determines whether a charge is classified...,What are the tax implications of forming a non...,1470544_1
3,gpt-4o-2024-11-20,402,43,What are the grounds for denying a Rule 3.850 ...,What are the qualifications to file for an evi...,4200405_1
4,gpt-4o-2024-11-20,5045,41,What are the legal considerations for withdraw...,What are the steps to file for divorce in Tenn...,1083484_2


## Confirm all datapoints were predicted & join prediction to original

In [13]:
assert len(train) == len(train_queries)

In [14]:
train_df = train.merge(train_queries, how="left", on="chunk_id")
train_df.head()

,chunk_id,chunk_size,chunked_opinion,opinion_id,opinion,model,input_tokens,output_tokens,relevant,irrelevant
0,4200405_1,197,DISTRICT COURT OF APPEAL OF THE STATE OF FLORI...,4200405,DISTRICT COURT OF APPEAL OF THE STATE OF FLOR...,gpt-4o-2024-11-20,402,43,What are the grounds for denying a Rule 3.850 ...,What are the qualifications to file for an evi...
1,2790710_1,1067,Twersky v Incorporated Vil. of Great Neck ( 20...,2790710,Twersky v Incorporated Vil. of Great Neck ( 2...,gpt-4o-2024-11-20,1211,43,What legal principles determine a property own...,What are the tax implications of selling a pro...
2,1470544_1,2286,951 A. 2d 180 ( 2008 ) Philip S. HORNER v. GOV...,1470544,951 A.2d 180 (2008) Philip S. HORNER v. GOVERN...,gpt-4o-2024-11-20,2492,48,What determines whether a charge is classified...,What are the tax implications of forming a non...
3,1083484_1,7760,IN THE COURT OF CRIMINAL APPEALS OF TENNESSEE ...,1083484,IN THE COURT OF CRIMINAL APPEALS OF TENNESSEE...,gpt-4o-2024-11-20,7965,41,What are the legal standards for withdrawing a...,What are the tax implications of starting a sm...
4,1083484_2,4820,"207 ). As pointed out above, the applicable st...",1083484,IN THE COURT OF CRIMINAL APPEALS OF TENNESSEE...,gpt-4o-2024-11-20,5045,41,What are the legal considerations for withdraw...,What are the steps to file for divorce in Tenn...


# Make GPT call on Val data

In [15]:
%%time

val_pred = util.predict(val)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 3
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 2
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 4
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 1
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 0
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 5
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 8
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Completed: 6
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 O

CPU times: user 4.75 s, sys: 244 ms, total: 4.99 s
Wall time: 26 s


In [16]:
val_queries = pd.DataFrame(val_pred)
val_queries.head()

,model,input_tokens,output_tokens,relevant,irrelevant,chunk_id
0,gpt-4o-2024-11-20,631,45,What is required to prove the uninsured operat...,What are the tax implications of owning a vehi...,6198051_1
1,gpt-4o-2024-11-20,1758,45,Can someone accused of murdering a testator be...,What are the tax implications for inheriting p...,5239595_1
2,gpt-4o-2024-11-20,3391,44,How do courts determine the validity of a will...,What are the rules for enforcing non-compete a...,3531217_1
3,gpt-4o-2024-11-20,2497,47,Can pension benefits accrued after a divorce b...,What are the tax implications of forming a lim...,1467030_1
4,gpt-4o-2024-11-20,893,39,What are the legal principles governing fraud ...,What are the legal implications of intellectua...,6237262_1


## Confirm all datapoints were predicted & join prediction to original

In [17]:
assert len(val) == len(val_queries)

In [18]:
val_df = val.merge(val_queries, how="left", on="chunk_id")
val_df.head()

,chunk_id,chunk_size,chunked_opinion,opinion_id,opinion,model,input_tokens,output_tokens,relevant,irrelevant
0,6237262_1,674,Mr. Justice Mercur delivered the opinion of th...,6237262,Mr. Justice Mercur delivered the opinion of t...,gpt-4o-2024-11-20,893,39,What are the legal principles governing fraud ...,What are the legal implications of intellectua...
1,1467030_1,2337,217 N. J. Super. 541 ( 1987 ) 526 A. 2d 290 AL...,1467030,217 N.J. Super. 541 (1987) 526 A.2d 290 ALAN C...,gpt-4o-2024-11-20,2497,47,Can pension benefits accrued after a divorce b...,What are the tax implications of forming a lim...
2,5239595_1,1603,"Howard, J. : By the ' will of Byron S. Briggs,...",5239595,"Howard, J.: By the'will of Byron S. Briggs, w...",gpt-4o-2024-11-20,1758,45,Can someone accused of murdering a testator be...,What are the tax implications for inheriting p...
3,6198051_1,410,"Alfred M. Lama, J. In this case, the defendant...",6198051,"Alfred M. Lama, J. In this case, the defendan...",gpt-4o-2024-11-20,631,45,What is required to prove the uninsured operat...,What are the tax implications of owning a vehi...
4,3531217_1,3227,The purpose of this proceeding is twofold. Fir...,3531217,The purpose of this proceeding is twofold. Fir...,gpt-4o-2024-11-20,3391,44,How do courts determine the validity of a will...,What are the rules for enforcing non-compete a...


# Save the data for future use

In [19]:
train_df.to_csv("outputs/5.finetune_train_8192.csv", index=False)
val_df.to_csv("outputs/5.finetune_val_8192.csv", index=False)